**重要：delta_GPA绝对不允许对本文件做任何改动**

In [ ]:
'''
这个Jupyter Notebook包含了基于组分贡献法（Component Contribution，CC）的用于预测自由能（Gibbs Free Energy，ΔG）的函数示例。
如果用户请求使用CC方法，对于简单的情形下的生化反应热力学分析，可以直接调用这些函数进行预测。

分为两大类：
- 生成自由能预测（Formation Energy，$Δ_f G$）
- 反应自由能预测（Reaction Energy，$Δ_r G$）
这些函数可以帮助用户在不同条件下预测自由能，从而支持生化反应的热力学分析
'''

In [1]:
'''
必要库，在调用本文档的函数之前请确保import这些库
'''
import subprocess
import json
import sys
import os
from typing import Optional, Tuple, Union

import numpy as np
import numpy.typing as npt
from rdkit import Chem, RDLogger

from equilibrator_api import ComponentContribution, Reaction, Q_

In [2]:
'''
初始化CC类，以便在后续函数中调用
'''
cc = ComponentContribution() # 初始化组分贡献法对象放在全局，避免重复初始化

## 生成自由能预测（Formation Energy，$Δ_f G'$）

这部分包含了用于计算**化合物生成自由能**的函数

In [3]:
def get_compound(identifier: str, cc) -> Optional[object]:
    """
    根据标识符获取化合物对象，按优先级尝试多种策略

    注意：调用本函数必须全局初始化cc = ComponentContribution()
    
    Args:
        identifier: 化合物标识符(支持InChI、KEGG、BIGG、Metacyc、SMILES等格式)
        - 格式1: InChI (以 "InChI=" 开头)
        - 格式2: KEGG (以 "C" 开头，后跟5位数字)
        - 格式3: BIGG (直接使用BIGG ID)
        - 格式4: Metacyc (直接使用Metacyc ID)
        - 格式5: SMILES (有效的SMILES字符串，最后尝试)
        cc: 已初始化的 ComponentContribution 对象
    
    Returns:
        成功时返回化合物对象，失败时返回 None
    """

    RDLogger.DisableLog('rdApp.error')  # 禁用RDKit错误日志输出

    def try_get_compound(query: str) -> Optional[object]:
        """尝试获取化合物，失败或返回None时返回None"""
        try:
            result = cc.get_compound(query)
            return result if result is not None else None
        except Exception:
            return None
    
    def is_smiles(s: str) -> bool:
        """判断字符串是否为有效的SMILES格式"""
        try:
            mol = Chem.MolFromSmiles(s)
            return mol is not None
        except Exception:
            return False
    
    def smiles_to_inchi(smiles: str) -> Optional[str]:
        """将SMILES转换为InChI"""
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is not None:
                inchi = Chem.MolToInchi(mol)
                return inchi
            return None
        except Exception:
            return None
    
    # 策略0: 直接查询（原始标识符）
    compound = try_get_compound(identifier)
    if compound is not None:
        return compound
    
    # 策略0.5: 通用搜索
    compound = cc.search_compound(identifier)
    if compound is not None:
        return compound

    
    # 策略1: InChI格式
    compound = cc.get_compound_by_inchi(identifier)
    if compound is not None:
        return compound


    # 策略2: KEGG格式 (C + 5位数字)
    compound = try_get_compound(f"kegg:{identifier}")
    if compound is not None:
        return compound
    
    # 策略3: BIGG数据库
    compound = try_get_compound(f"bigg.metabolite:{identifier}")
    if compound is not None:
        return compound
    
    # 策略4: MetaCyc数据库
    compound = try_get_compound(f"metacyc.compound:{identifier}")
    if compound is not None:
        return compound
    
    # 策略5: SMILES格式（最后尝试，因为检测成本较高）
    if is_smiles(identifier):
        try:
            inchi = smiles_to_inchi(identifier)
            if inchi:
                compound = cc.get_compound_by_inchi(inchi)
                if compound is not None:
                    return compound
        except Exception:
            pass
    
    # 所有策略均失败
    return None

In [4]:
# 示例调用
get_compound("phophate", cc)

Compound(id=12, inchi_key=NBIIXXVUZAFLBC-UHFFFAOYSA-L)

In [5]:
def standard_dgf_prime_CC(
    cpd: str, 
    cc: ComponentContribution,
    p_h: float = 7.0, 
    p_mg: float = 3.0, 
    I: float = 0.25, 
    T: float = 298.15,
    physiological: bool = False # 是否转换为1mM生理浓度
) -> Tuple[np.floating, np.floating]:
    '''
    使用组分贡献法(Component Contribution)计算化合物的变换生成自由能 (Δ_fG'°或Δ_fG'm)

    注意：调用本函数必须提前全局初始化cc = ComponentContribution()，并调用get_compound()获取化合物对象。
    
    参数:
    cpd: 化合物名称、InChI 或 KEGG ID
    cc: 已初始化的 ComponentContribution 对象
    physiological: 如果为 True，返回 1mM 浓度下的结果；
                   如果为 False (默认)，返回 1M 标准态结果。
    
    返回:
    (能量值, 误差值) 单位: kJ/mol
    '''
    try:
        # 设置条件
        cc.p_h = Q_(p_h)
        cc.p_mg = Q_(p_mg)
        cc.ionic_strength = Q_(f'{I}M')
        cc.temperature = Q_(f'{T}K')

        # 获取化合物
        compound = get_compound(cpd, cc)
        if compound is None:
            raise ValueError(f"无法找到化合物: {cpd}")

        # 创建虚拟反应: 0 -> 1 化合物
        rxn_c = Reaction({compound: 1})
        
        # 计算能量
        dg_prime_measurement = cc.standard_dg_prime(rxn_c)
        
        # 提取数值
        val = dg_prime_measurement.value.m_as("kJ/mol")
        err = dg_prime_measurement.error.m_as("kJ/mol")
        
        # 如果需要生理浓度 (1mM)，加上修正项
        if physiological:
            from scipy.constants import R # J/(K·mol)
            R = R * 1e-3  # kJ/(K·mol)
            correction = R * T * np.log(1e-3)
            val += correction
        
        return np.float64(val), np.float64(err)
    
    except Exception as e:
        print(f"处理化合物 {cpd} 时出错: {str(e)}")
        return np.nan, np.nan

In [6]:
# 示例调用
dGf, std = standard_dgf_prime_CC('glucose', cc) # 注意输入化合物名称时一定使用小写
print(f"Glucose Δ_fG'° (Standard conditions): {dGf:.2f} ± {std:.2f} kJ/mol")

Glucose Δ_fG'° (Standard conditions): -426.53 ± 0.64 kJ/mol


In [7]:
# 示例调用
dGf, std = standard_dgf_prime_CC('kegg:C00002', cc, p_h=8, p_mg=3, I=0.3, physiological=True)
print(f"ATP Δ_fG'^m (pH=8, pMg=3, I=0.3, physiological): {dGf:.2f} ± {std:.2f} kJ/mol")

ATP Δ_fG'^m (pH=8, pMg=3, I=0.3, physiological): -2246.12 ± 1.49 kJ/mol


## 反应自由能预测（Reaction Energy，$Δ_r G'$）

这部分包含了用于计算**生化反应自由能**的函数

注意：如果此部分函数运行失效，可以在生成自由能预测函数的基础上重新编写反应自由能预测函数，在工作区运行

In [8]:
def standard_dgr_prime_CC(
    rxn: str,
    cc: ComponentContribution,
    p_h: float = 7.0, 
    p_mg: float = 3.0, 
    I: float = 0.25, 
    T: float = 298.15,
    physiological: bool = False # 是否转换为1mM生理浓度
) -> Tuple[np.floating, np.floating, np.floating]:
    """
    使用组分贡献法(Component Contribution)计算反应的变换生成自由能 (Δ_rG'°或Δ_rG'^m)和平衡常数 (K')

    注意：调用本函数必须提前全局初始化cc = ComponentContribution()
    
    参数:
    rxn: 反应式字符串
    cc: 已初始化的 ComponentContribution 对象
    physiological: 如果为 True，返回 1mM 浓度下的结果；
                   如果为 False (默认)，返回 1M 标准态结果。
    
    返回:
    (能量值, 误差值, 平衡常数) 单位: kJ/mol    
    """
    try:        
        # 设置条件
        cc.p_h = Q_(p_h)
        cc.p_mg = Q_(p_mg)
        cc.ionic_strength = Q_(f'{I}M')
        cc.temperature = Q_(f'{T}K')
        
        # 解析反应
        reaction = cc.parse_reaction_formula(rxn)
        
        # 计算反应的 ΔG'°
        dg_prime_measurement = cc.standard_dg_prime(reaction)

        # 提取数值
        val = dg_prime_measurement.value.m_as("kJ/mol")
        err = dg_prime_measurement.error.m_as("kJ/mol")

        from scipy.constants import R # J/(K·mol)
        R = R * 1e-3  # kJ/(K·mol)

        if physiological:
            lnQ = 0
            for compound, coeff in reaction.sparse.items():
                lnQ += coeff * np.log(1e-3)

            # 应用浓度校正
            val = val + R * T * lnQ
        
        # 计算平衡常数
        RT = R * T
        K_prime = np.exp(-val / RT)
        
        return np.float64(val), np.float64(err), np.float64(K_prime)
    
    except Exception as e:
        print(f"处理反应 {rxn} 时出错: {str(e)}")
        return np.nan, np.nan, np.nan

In [9]:
# 示例调用
dGr, std, K_prime = standard_dgr_prime_CC('kegg:C00294 + kegg:C00009 = kegg:C00262 + kegg:C00620', cc)
print(f"Inosine phosphorolysis Δ_rG'° (Standard conditions): {dGr:.2f} ± {std:.2f} kJ/mol, K': {K_prime:.2e}")

Inosine phosphorolysis Δ_rG'° (Standard conditions): 8.81 ± 1.07 kJ/mol, K': 2.86e-02


In [10]:
# 示例调用
dGr, std, K_prime = standard_dgr_prime_CC(
    'atp + creatine = adp + phosphocreatine', # 注意输入化合物名称时一定使用小写
    cc,
    p_h=8.0, 
    p_mg=3.0, 
    I=0.3, 
    physiological=True
)
print(f"Creatine phosphorylation Δ_rG'^m (pH=8, pMg=3, I=0.3, T=298.15K, physiological): {dGr:.2f} ± {std:.2f} kJ/mol, K': {K_prime:.2e}")

Creatine phosphorylation Δ_rG'^m (pH=8, pMg=3, I=0.3, T=298.15K, physiological): 9.13 ± 0.21 kJ/mol, K': 2.51e-02


In [11]:
def example_reaction_with_different_concentrations(
    rxn: str,
    cc: ComponentContribution,
    p_h: float = 7.0, 
    p_mg: float = 3.0, 
    I: float = 0.25, 
    T: float = 298.15,
    ) -> np.floating:
    """
    使用组分贡献法(Component Contribution)，为不同物质设置不同浓度来计算非标准条件下的反应自由能(ΔG')的例子，对于具体情况，需要修改对应的化合物字典

    注意：调用本函数必须提前全局初始化cc = ComponentContribution()
    
    参数:
    rxn: 反应式字符串
    cc: 已初始化的 ComponentContribution 对象

    返回:
    实际能量值 单位: kJ/mol
    """

    # 设置条件
    cc.p_h = Q_(p_h)
    cc.p_mg = Q_(p_mg)
    cc.ionic_strength = Q_(f'{I}M')
    cc.temperature = Q_(f'{T}K')

    # 解析反应
    reaction = cc.parse_reaction_formula(rxn)
    
    print(f'反应: {rxn}')
    print('反应物和产物的化学计量系数:')
    for compound, coeff in reaction.sparse.items():
        # 尝试获取化合物名称
        compound_name = "Unknown"
        for identifier in compound.identifiers:
            if identifier.registry.namespace in ['kegg', 'bigg.metabolite', 'chebi', 'hmdb']:
                compound_name = identifier.accession
                break
        print(f'  {compound_name}: {coeff}')
    
    # 计算标准条件下的ΔG'°
    dg_prime_standard = cc.standard_dg_prime(reaction)
    dg_std_kjmol = dg_prime_standard.value.m_as('kJ/mol')
    
    print(f'\n标准条件下的ΔG\'°: {dg_std_kjmol:.2f} kJ/mol')
    
    # 设置代谢物浓度
    phys_conc = {
        'ins': 5e-3,        # 5 mM
        'pi': 1e-3,           # 1 mM
        'hxan': 0.1e-3,  # 0.1 mM
        'r1p': 0.1e-3          # 0.1 mM
    }
    
    # 创建一个浓度字典，键是compound对象，值是浓度（单位M）
    concentrations = {}
    for compound, coeff in reaction.sparse.items():
        # 获取化合物名称
        compound_name = "Unknown"
        for identifier in compound.identifiers:
            if identifier.registry.namespace in ['kegg', 'bigg.metabolite', 'chebi', 'hmdb']:
                compound_name = identifier.accession.lower()
                break

        name_lower = compound_name.lower()
        if 'ins' in name_lower:
            concentrations[compound] = phys_conc['ins']
        elif 'pi' in name_lower:
            concentrations[compound] = phys_conc['pi']
        elif 'hxan' in name_lower:
            concentrations[compound] = phys_conc['hxan']
        elif 'r1p' in name_lower:
            concentrations[compound] = phys_conc['r1p']
        else:
            concentrations[compound] = 1e-3  # 默认浓度
    
    print('\n各物质的浓度设置:')
    for compound, conc in concentrations.items():
        # 获取化合物名称
        compound_name = "Unknown"
        for identifier in compound.identifiers:
            if identifier.registry.namespace in ['kegg', 'bigg.metabolite', 'chebi', 'hmdb']:
                compound_name = identifier.accession
                break
        print(f'  {compound_name}: {conc} M ({conc*1000:.3f} mM)')
    
    # 计算反应商 (Q)
    lnQ = 0
    for compound, coeff in reaction.sparse.items():
        conc = concentrations[compound]
        lnQ += coeff * np.log(conc)  # coeff是化学计量系数
    
    print(f'\n反应商的对数 ln(Q): {lnQ:.4f}')
    
    # 计算非标准条件下的ΔG
    from scipy.constants import R # J/(K·mol)
    RT = R * 1e-3 * T  # R*T，转换为kJ/mol单位
    dg_non_standard = dg_std_kjmol + RT * lnQ
    
    print(f'\n非标准条件下的ΔG\' : {dg_non_standard:.2f} kJ/mol')
    print(f'标准条件到非标准条件的校正: {RT * lnQ:.2f} kJ/mol')
    
    # 计算反应的平衡常数
    K_prime = np.exp(-dg_std_kjmol / RT)
    print(f'\n平衡常数 K\' : {K_prime:.2e}')
    
    # 计算反应的驱动力（使用实际浓度）
    Q_actual = np.exp(lnQ)
    delta_g_actual = dg_std_kjmol + RT * lnQ
    print(f'实际反应商 Q = {Q_actual:.2e}')
    print(f'实际驱动力 ΔG\' = {delta_g_actual:.2f} kJ/mol')
    
    # 判断反应方向
    if delta_g_actual < 0:
        print('在这些条件下，反应倾向于从左到右进行')
    elif delta_g_actual > 0:
        print('在这些条件下，反应倾向于从右到左进行')
    else:
        print('在这些条件下，反应处于平衡状态')

    return delta_g_actual

In [12]:
# 示例调用
dGr = example_reaction_with_different_concentrations('kegg:C00294 + kegg:C00009 = kegg:C00262 + kegg:C00620' , cc)

print(f'\n计算得到的非标准条件下的ΔG\' : {dGr:.2f} kJ/mol')

反应: kegg:C00294 + kegg:C00009 = kegg:C00262 + kegg:C00620
反应物和产物的化学计量系数:
  ins: -1.0
  pi: -1.0
  hxan: 1.0
  r1p: 1.0

标准条件下的ΔG'°: 8.81 kJ/mol

各物质的浓度设置:
  ins: 0.005 M (5.000 mM)
  pi: 0.001 M (1.000 mM)
  hxan: 0.0001 M (0.100 mM)
  r1p: 0.0001 M (0.100 mM)

反应商的对数 ln(Q): -6.2146

非标准条件下的ΔG' : -6.60 kJ/mol
标准条件到非标准条件的校正: -15.41 kJ/mol

平衡常数 K' : 2.86e-02
实际反应商 Q = 2.00e-03
实际驱动力 ΔG' = -6.60 kJ/mol
在这些条件下，反应倾向于从左到右进行

计算得到的非标准条件下的ΔG' : -6.60 kJ/mol
